In [1]:
import pandas as pd
import math
from sklearn.preprocessing import normalize
from collections import defaultdict
import operator
import os
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import gc

In [2]:
def tftoidf(word_dict,cut_code,word_idf):
    ans={}
   
    for i in word_dict:
        ans[i]=(cut_code[word_dict[i]])*word_idf[i]
    return ans

def compare2(s1_cut,s2_cut,DF,co):
   
   
    word_set = set(s1_cut).union(set(s2_cut))
    word_dict = dict()
    i = 0
   
    for word in word_set:#比較標號
        word_dict[word] = i
        i += 1
    
    #--------TF
    s1_cut_code = [word_dict[word] for word in s1_cut]#TF
    s1_cut_code = [0]*len(word_dict)

    for word in s1_cut:#S1詞頻
        s1_cut_code[word_dict[word]]+=1
    
    s2_cut_code = [word_dict[word] for word in s2_cut]

    s2_cut_code = [0]*len(word_dict)
    for word in s2_cut:#S2詞頻
        s2_cut_code[word_dict[word]]+=1
    for i in range(len(word_dict)):
        s1_cut_code[i]=s1_cut_code[i]/len(word_dict)
        s2_cut_code[i]=s2_cut_code[i]/len(word_dict)
    #--------IDF1

     
    doc_num=1000
    word_idf={}
    word_doc=DF
    s1_idf_code = s1_cut #S1單詞

    for i in word_dict:
        word_idf[i]=math.log(doc_num/(word_doc[i]+1))
    #print(word_idf)

    #--------IDF2.

    doc_num=1000
    word_idf2={}
    word_doc=DF
    s2_idf_code = s2_cut

    for i in word_dict:
        word_idf2[i]=math.log(doc_num/(word_doc[i]+1))
    #--------TF*IDF.  

    word_tf_idf=tftoidf(word_dict,s1_cut_code,word_idf)
   
    word_tf_idf2=tftoidf(word_dict,s2_cut_code,word_idf2)

    # 计算余弦相似度

    sum = 0
    sq1 = 0
    sq2 = 0
    for i in word_dict:
        sum += word_tf_idf[i] * word_tf_idf2[i]
        #print(i)
        sq1 += pow(word_tf_idf[i], 2)
        sq2 += pow(word_tf_idf2[i], 2)

    try:
        result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 5)
    except ZeroDivisionError:
        result = 0.0
    if co%10000==0:
        del word_tf_idf
        del word_tf_idf2
        del word_set
        del word_dict
        gc.collect()
    return result

# count documents

In [3]:
documentPath = 'documents/'
documentList = os.listdir(documentPath)
documentnum=[]
dolist=[]
for file in documentList:
    f = open(documentPath+file, 'r')
    dolist.append(f.read())
    file=file.replace('.txt','')
    documentnum.append(file)
setdolist=[]   
col=[]
col.append('name')
col.append('cut')
documents_test = pd.DataFrame(columns=col,dtype=float)

documents_test['name']=documentnum
for i in range(len(documents_test)):
    documents_test['cut'][i]=dolist[i].split(' ')
documents_test

/var/folders/ks/dqhrwlns2r1506smq16l4dth0000gn/T/ipykernel_70839/2539168238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_test['cut'][i]=dolist[i].split(' ')


,name,cut
0,D744477,"[answerscom, ®, wiki, answers, ®, categories, ..."
1,D2616256,"[richard, walker, home, meaning, of, richard, ..."
2,D1744040,[tncolumbia]
3,D2696324,"[country, inn, , suites, by, radisson, st, pet..."
4,D932852,"[jewelry, metals, 101, gold, silver, and, plat..."
...,...,...
995,D2117450,"[omc, cobra, king, cobra, toll, free, , 877283..."
996,D2886245,"[hotaru, shidare, anime, manga, kanji枝垂, ほたるro..."
997,D810432,"[home, , protein, , silk, cocoon, protein, pow..."
998,D2984334,"[introduction, the, station, was, established,..."


In [4]:
DF = {} 
for i in range(len(documents_test)): 
    tokens = documents_test['cut'][i] 
    for w in tokens: 
        try: 
            DF[w].add(i) 
        except: 
            DF[w] = {i}         
for i in DF:
    DF[i]=len(DF[i])

# count query

In [5]:
queriesPath = 'queries/'
queriesList = os.listdir(queriesPath)
qlist=[]
namelist=[]
af = open('queries_id_list.txt', 'r')
for file in af:
    file=file.replace('\n','')
    namelist.append(file)
    f = open(queriesPath+file+'.txt', 'r')
    qlist.append(f.read())   
    
col=[]
col.append('name')
col.append('cut')
f_test = pd.DataFrame(columns=col,dtype=float)
f_test['name']=namelist
for i in range(len(f_test)):
    f_test['cut'][i]=qlist[i].split(' ')
f_test

/var/folders/ks/dqhrwlns2r1506smq16l4dth0000gn/T/ipykernel_70839/3267628015.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_test['cut'][i]=qlist[i].split(' ')


,name,cut
0,1001149,"[where, the, odontoid, process, is, found]"
1,1008059,"[which, cranial, nerve, leaves, the, cranial, ..."
2,1037699,"[who, is, richard, walker]"
3,1150201,"[what, is, the, minimum, wage, in, los, angele..."
4,1158307,"[what, is, a, good, gluten, free, dinner]"
5,1170700,"[side, effects, for, diastat]"
6,1171932,"[pivotal, meaning]"
7,12567,"[after, how, many, years, does, a, car, seat, ..."
8,167717,"[does, it, cost, more, to, build, up, or, out]"
9,176461,"[dull, pain, in, left, side, under, ribs]"


In [8]:
final=[]
score=[]
co=0
for i in range(len(f_test)):
    key_value ={}     
    for j in range(len(documents_test)):
        ans=compare2(f_test['cut'][i],documents_test['cut'][j],DF,co)
        key_value[documents_test['name'][j]]=ans
        co+=1
    score.append(sorted(key_value.items(), key = lambda kv:(kv[1], kv[0]),reverse=True))
col=[]
col.append('Query')
col.append('RetrievedDocuments')
answer = pd.DataFrame(columns=col,dtype=float)
answer['Query']=namelist
answer['RetrievedDocuments']=score

In [9]:
answer

,Query,RetrievedDocuments
0,1001149,"[(D398561, 0.19835), (D1678969, 0.11435), (D95..."
1,1008059,"[(D101326, 0.41559), (D2932274, 0.15744), (D53..."
2,1037699,"[(D2616256, 0.90773), (D2240480, 0.25255), (D3..."
3,1150201,"[(D1193447, 0.6309), (D1675704, 0.53219), (D12..."
4,1158307,"[(D540613, 0.38997), (D1585045, 0.07808), (D39..."
5,1170700,"[(D280716, 0.89907), (D160182, 0.05509), (D207..."
6,1171932,"[(D214261, 0.49421), (D442273, 0.03608), (D333..."
7,12567,"[(D279470, 0.47967), (D3232522, 0.16273), (D15..."
8,167717,"[(D3527133, 0.3342), (D1292165, 0.28627), (D33..."
9,176461,"[(D3070021, 0.21057), (D62037, 0.1733), (D3075..."


In [10]:
answer1=answer.copy()

In [11]:
for i in range(len(answer1)):
    st=''
    for j in range(len(answer1['RetrievedDocuments'][i])):
        if(j==999):
            st=st+answer1['RetrievedDocuments'][i][j][0]
        else:
            st=st+answer1['RetrievedDocuments'][i][j][0]+' '
    answer1['RetrievedDocuments'][i]=st
answer1

,Query,RetrievedDocuments
0,1001149,D398561 D1678969 D954322 D2117298 D1304667 D17...
1,1008059,D101326 D2932274 D536378 D2760316 D1233108 D10...
2,1037699,D2616256 D2240480 D3103838 D1727368 D612714 D5...
3,1150201,D1193447 D1675704 D1234215 D1675096 D1927959 D...
4,1158307,D540613 D1585045 D396774 D512532 D2026282 D216...
5,1170700,D280716 D160182 D2072010 D3075569 D1068252 D31...
6,1171932,D214261 D442273 D3330476 D1674095 D998007 D579...
7,12567,D279470 D3232522 D1556006 D714459 D442910 D103...
8,167717,D3527133 D1292165 D3324553 D2813899 D649109 D2...
9,176461,D3070021 D62037 D3075569 D564273 D3065831 D327...


In [12]:
answer1.to_csv('submission.csv',index=0)